In [ ]:
notebook_type = "kaggle"

# Импорты и константы

In [2]:
if notebook_type == "kaggle":
    main_path = "/kaggle/"
    kaggle = True
else:
    main_path = ""
    kaggle = False

In [3]:
destination_path = f"{main_path}working/"
destination_input = f"{main_path}input/"
check_data = ""

In [4]:
if kaggle:
    !pip install -q mediapipe

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires cloudpickle~=2.2.1, but you have cloudpickle 3.0.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires protobuf<4,>3.12.2, but you have protobuf 4.25.5 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 17.0.0 which is incompatible.
google-cloud-aiplatform 0.6.0a1 requires google-api-core[grpc]<2.0.0dev,>=1.22.2, but you have google-api-core 2.11.1 which is incompatible.
google-cloud-automl 1.0.1 requires google-api-core[grpc]<2.0.0dev,>=1.14.0, but you have google-api-core 2.11.1 which is incompatible.
google-cloud-bigquery 2.34.4 requires protobuf<

In [5]:
if kaggle:
    !wget -O pose_landmarker_heavy.task -q https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/latest/pose_landmarker_heavy.task
    !wget -O pose_landmarker_lite.task -q https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_lite/float16/latest/pose_landmarker_lite.task
    !wget -O pose_landmarker_full.task -q https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_full/float16/latest/pose_landmarker_full.task

In [ ]:
import gc
import hashlib
import time
import io
import random

import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
import traceback
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms as T
from torchvision.models.optical_flow import Raft_Small_Weights, raft_small
from torchvision.utils import flow_to_image
from tqdm import tqdm
from PIL import Image
import mediapipe as mp

In [12]:
def set_seed(seed):
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

In [13]:
torch.cuda.is_available()

False

In [14]:
# Настройка устройства (GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# Функции расчета данных

In [15]:
def get_hash_key(params):
    """
    Генерация хэша на основе кортежа параметров
    :params: tuple — параметры, которые используются для генерации эмбеддингов
    """
    # Превращаем параметры в строку и генерируем хэш с использованием hashlib
    params_string = str(params).encode("utf-8")
    return hashlib.md5(params_string).hexdigest()

### Класс расчета скелетных данных на основе MediaPipe

In [16]:
# # Выбор модели для определения скелетных данных (lite или large модель)
# if kaggle:
#     model_path = "/kaggle/working/pose_landmarker_lite.task"
# else:
#     model_path = "../../../resources/pose_landmarker_lite.task"

In [17]:
# # Выбор модели для определения скелетных данных (lite или large модель)
# if kaggle:
#     model_path = "/kaggle/working/pose_landmarker_heavy.task"
# else:
#     model_path = "../../../resources/pose_landmarker_heavy.task"

In [18]:
# Выбор модели для определения скелетных данных (lite или large модель)
if kaggle:
    model_path = "/kaggle/working/pose_landmarker_full.task"
else:
    model_path = "../../../resources/pose_landmarker_full.task"

In [19]:
BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

class SFeatureProcessor:
    def __init__(
        self,
        model_path,
        video_idx,
        new_width=640,
        new_height=360,
        display=False,
        calculate_masks=True,
        do_resize=False,
    ):
        self.model_path = model_path
        self.new_width = new_width
        self.new_height = new_height
        self.width = None
        self.height = None
        self.display = display
        self.calculate_masks = calculate_masks
        self.do_resize = do_resize
        self.buffer = []  # Буфер для сохранения данных
        self._initialize_detector()  # Инициализация детектора
        self.video_idx = video_idx

    def _initialize_detector(self):
        """Инициализация или повторная инициализация детектора."""
        self.options = PoseLandmarkerOptions(
            base_options=BaseOptions(model_asset_path=self.model_path),
            running_mode=VisionRunningMode.VIDEO,
            output_segmentation_masks=self.calculate_masks,  # Включаем или отключаем расчет масок сегментации
        )
        self.detector = PoseLandmarker.create_from_options(self.options)

    def process_frame(self, frame, timestamp_ms):
        # Обработка кадра (при необходимости изменяем его размер)
        if self.do_resize:
            frame_resized = cv2.resize(frame, (self.new_width, self.new_height))
        else:
            frame_resized = frame

        self.width = frame_resized.shape[0]
        self.height = frame_resized.shape[1]
        frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)

        # Обработка кадра и получение результата синхронно
        detection_result = self.detector.detect_for_video(mp_image, timestamp_ms)

        self.result_callback(detection_result, timestamp_ms)

        return frame_resized  # Возвращаем измененный кадр для отображения (если было применено resize)

    def result_callback(self, detection_result, timestamp_ms):
        """Обработка результатов детекции для каждого кадра."""
        pose_landmarks_list = detection_result.pose_landmarks
        pose_world_landmarks_list = detection_result.pose_world_landmarks
        segmentation_masks = detection_result.segmentation_masks if self.calculate_masks else None

        if pose_landmarks_list and pose_world_landmarks_list:
            for pose_landmarks, pose_world_landmarks in zip(pose_landmarks_list, pose_world_landmarks_list):
                landmarks_array = np.array([[landmark.x, landmark.y, landmark.z] for landmark in pose_landmarks], dtype=np.float32)
                world_landmarks_array = np.array([[landmark.x, landmark.y, landmark.z] for landmark in pose_world_landmarks], dtype=np.float32)

                mask_array = segmentation_masks[0].numpy_view().astype(np.float32) if self.calculate_masks else None

                # Сохраняем данные в буфер
                self.buffer.append((timestamp_ms, landmarks_array, world_landmarks_array, mask_array))
        else:
            landmarks_array = np.zeros((33, 3), dtype=np.float32)
            world_landmarks_array = np.zeros((33, 3), dtype=np.float32)
            mask_array = (np.zeros((self.width, self.height), dtype=np.float32) if self.calculate_masks else None)

            # Сохраняем пустые данные в буфер
            self.buffer.append((timestamp_ms, landmarks_array, world_landmarks_array, mask_array))

    def return_data(self):
        """Возвращает собранные данные из буфера."""
        if self.buffer:
            landmarks_data = np.array([item[1] for item in self.buffer])
            world_landmarks_data = np.array([item[2] for item in self.buffer])
            masks_data = np.array([item[3] for item in self.buffer if item[3] is not None]) if self.calculate_masks else None
            self.buffer.clear()

            return (landmarks_data, world_landmarks_data, masks_data)

    def process_video(self, frames, fps):
        """Обработка видео и извлечение данных по каждому кадру."""
        frame_idx = 0

        for frame in frames:
            # Рассчитываем временную метку для каждого кадра
            timestamp_ms = int((frame_idx / fps) * 1000)  
            self.process_frame(frame, timestamp_ms)
            frame_idx += 1

        # Сохранение всех данных после завершения обработки
        landmarks_data, world_landmarks_data, masks_data = self.return_data()

        # Освобождаем ресурсы
        self.cleanup()

        return (landmarks_data, world_landmarks_data, masks_data)

    def cleanup(self):
        """Метод для очистки и освобождения ресурсов."""
        self.buffer.clear()  # Очищаем буфер данных
        del self.detector  # Удаляем детектор
        gc.collect()  # Принудительный запуск сборщика мусора для освобождения памяти


### Функции корректировка скелетных координат

In [20]:
def check_landmarks(landmarks_data, world_landmarks_data, figure_masks_data):
    """
    Проверяет каждый кадр в массиве landmarks_data. Если в кадре найдены значения,
    которые не входят в диапазон (0, 1), этот кадр и соответствующий кадр в 
    world_landmarks_data обнуляются.

    Если более 50% кадров последовательности или более 10 кадров подряд обнулены, 
    то такие данные помечаются, как не валидные.
    
    Функция также возвращает маску валидности последовательности.
    """
    num_frames = landmarks_data.shape[0]
    validation_mask = np.ones((num_frames,), dtype=np.int32)  # Инициализируем маску валидности последовательности

    invalid_count = 0  # Счетчик невалидных кадров
    max_consecutive_invalid = 0  # Максимальное количество подряд идущих невалидных кадров
    current_consecutive_invalid = 0  # Текущая последовательность невалидных кадров

    for i in range(num_frames):
        # Проверяем, есть ли в кадре значения x или y за пределами (0, 1)
        if not np.all((landmarks_data[i, :, 0:2] > 0) & (landmarks_data[i, :, 0:2] < 1)):
            # Обнуляем кадр в обоих массивах
            landmarks_data[i] = np.zeros_like(landmarks_data[i])
            world_landmarks_data[i] = np.zeros_like(world_landmarks_data[i])
            figure_masks_data[i] = np.zeros_like(figure_masks_data[i])  # Оставляем, как есть - маска для других целей

            validation_mask[i] = 0  # Отмечаем кадр как невалидный
            invalid_count += 1
            current_consecutive_invalid += 1
            max_consecutive_invalid = max(max_consecutive_invalid, current_consecutive_invalid)
        else:
            current_consecutive_invalid = 0

    # Проверяем, валидна ли последовательность по количеству невалидных кадров
    is_valid_sequence = not (invalid_count > num_frames * 0.1 or max_consecutive_invalid >= 10)
    is_valid_sequence = not (invalid_count > num_frames * 0.25)
    is_valid_sequence = True
    is_valid_sequence = not (invalid_count == num_frames)
    
    return landmarks_data, world_landmarks_data, figure_masks_data, is_valid_sequence, validation_mask


In [21]:
def padding_landmarks(landmarks, crop_area, image_shape):
    min_x, min_y, max_x, max_y = crop_area
    width, height = image_shape
    # Корректируем координаты скелета относительно нового кропа и масштабируем
    adjusted_landmarks = []
    for landmark in landmarks:
        x, y, z = landmark
        x_adjusted = (x * width - min_x) / (max_x - min_x)  # Корректировка x
        y_adjusted = (y * height - min_y) / (max_y - min_y)  # Корректировка y
        adjusted_landmarks.append([x_adjusted, y_adjusted, z])
    return np.array(adjusted_landmarks)

### Обрезка изображения

In [23]:
def make_divisible_by_8(value):
    """Корректируем значение, чтобы оно стало кратным 8."""
    return (value + 7) // 8 * 8


def get_global_crop_area(landmarks_data, image_shape, padding=100):
    width, height = image_shape

    # Преобразуем координаты из диапазона [0, 1] в реальные координаты на изображении
    scaled_landmarks = landmarks_data.copy()
    scaled_landmarks[:, :, 0] *= width  # Масштабируем x координаты
    scaled_landmarks[:, :, 1] *= height  # Масштабируем y координаты

    # Фильтруем только ненулевые координаты
    valid_x = scaled_landmarks[:, :, 0][scaled_landmarks[:, :, 0] != 0]
    valid_y = scaled_landmarks[:, :, 1][scaled_landmarks[:, :, 1] != 0]

    # Проверяем, есть ли хоть одна ненулевая координата
    if len(valid_x) == 0 or len(valid_y) == 0:
        raise ValueError("No valid landmarks found within the range.")

    # Находим минимальные и максимальные координаты x и y для всех точек скелета на всех кадрах
    min_x = np.min(valid_x)
    max_x = np.max(valid_x)
    min_y = np.min(valid_y)
    max_y = np.max(valid_y)

    # Добавляем отступы (padding) вокруг найденной области
    min_x = int(min_x - padding)
    max_x = int(max_x + padding)
    min_y = int(min_y - padding)
    max_y = int(max_y + padding)

    # Ограничиваем значения в пределах размеров изображения
    min_x = max(0, min_x)
    max_x = min(width, max_x)
    min_y = max(0, min_y)
    max_y = min(height, max_y)

    # Вычисляем ширину и высоту области
    crop_width = max_x - min_x
    crop_height = max_y - min_y

    # Корректируем ширину и высоту, чтобы они были кратны 8
    crop_width = make_divisible_by_8(crop_width)
    crop_height = make_divisible_by_8(crop_height)

    # Определяем сторону квадрата как максимальную из высоты и ширины области
    square_size = max(crop_width, crop_height)

    # Центрируем квадрат относительно найденной области
    center_x = (min_x + max_x) // 2
    center_y = (min_y + max_y) // 2

    # Вычисляем координаты квадратного кропа
    # square_min_x = max(0, center_x - square_size // 2)
    # square_max_x = min(width, center_x + square_size // 2)
    # square_min_y = max(0, center_y - square_size // 2)
    # square_max_y = min(height, center_y + square_size // 2)
    
    square_min_x = center_x - square_size // 2
    square_max_x = center_x + square_size // 2
    square_min_y = center_y - square_size // 2
    square_max_y = center_y + square_size // 2

    return (square_min_x, square_min_y, square_max_x, square_max_y), (crop_width, crop_height)

### Функция добавления паддинга 

In [24]:
def crop_with_padding(images, crop_area):
    """
    Выполняет кроп изображений с учетом паддинга и возвращает бинарную маску, 
    где паддинг помечен 0, а изображение - 1.
    """
    min_x, min_y, max_x, max_y = crop_area
    height, width = images[0].shape[:2]  # Получаем размер первого изображения/маски

    # Проверяем, выходит ли квадрат за пределы изображения
    pad_left = max(0, -min_x)
    pad_right = max(0, max_x - width)
    pad_top = max(0, -min_y)
    pad_bottom = max(0, max_y - height)

    # Обновляем границы кропа, чтобы они не выходили за пределы изображения
    min_x = max(0, min_x)
    max_x = min(width, max_x)
    min_y = max(0, min_y)
    max_y = min(height, max_y)

    # Создаем массив для хранения обрезанных изображений с паддингом
    padded_images = []
    crop_masks = []

    # Обрабатываем каждое изображение в массиве
    for image in images:
        # Обрезаем изображение в соответствии с заданными границами
        cropped_image = image[min_y:max_y, min_x:max_x]

        # Создаем черный холст для изображения и бинарную маску
        if len(image.shape) == 3 and image.shape[2] == 3:
            # Цветное изображение (RGB)
            padded_image = np.zeros(
                (crop_area[3] - crop_area[1], crop_area[2] - crop_area[0], 3),
                dtype=np.uint8,
            )
        else:
            # Одноканальная маска (грейскейл)
            padded_image = np.zeros(
                (crop_area[3] - crop_area[1], crop_area[2] - crop_area[0]),
                dtype=np.uint8,
            )

        # Создаем маску той же формы, что и холст, и заполняем ее 0
        mask = np.zeros(
            (crop_area[3] - crop_area[1], crop_area[2] - crop_area[0]),
            dtype=np.uint8,
        )

        # Определяем, куда вставить обрезанное изображение на холст
        insert_x = pad_left
        insert_y = pad_top

        # Вставляем обрезанное изображение на холст с учетом паддинга
        padded_image[
            insert_y:insert_y + cropped_image.shape[0],
            insert_x:insert_x + cropped_image.shape[1]
        ] = cropped_image

        # Вставляем единицы в маску в тех же координатах
        mask[
            insert_y:insert_y + cropped_image.shape[0],
            insert_x:insert_x + cropped_image.shape[1]
        ] = 1

        # Добавляем изображение/маску с паддингом и маску в результирующий массив
        padded_images.append(padded_image)
        crop_masks.append(mask)

    # Преобразуем списки обратно в массивы
    return np.array(padded_images), np.array(crop_masks)


### Фунции расчета оптического потока

In [25]:
# Загрузка модели RAFT
model_raft = raft_small(weights=Raft_Small_Weights.C_T_V1.transforms).to(device)
model_raft.eval();

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Raft_Small_Weights.C_T_V2`. You can also use `weights=Raft_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/raft_small_C_T_V2-01064c6d.pth" to /root/.cache/torch/hub/checkpoints/raft_small_C_T_V2-01064c6d.pth
100%|██████████| 3.82M/3.82M [00:00<00:00, 33.6MB/s]


In [26]:
# Функция для масштабирования изображения и расчета коэффициентов масштаба
def resize_frame_with_scale(frame, target_size):
    """Масштабирование кадра и вычисление коэффициентов масштаба."""
    original_height, original_width = frame.shape[:2]
    resized_frame = cv2.resize(frame, target_size)

    # Коэффициенты масштабирования по осям X и Y
    scale_x = original_width / target_size[0]
    scale_y = original_height / target_size[1]

    return resized_frame, scale_x, scale_y


In [27]:
# Функция для расчета оптического потока
def calculate_optical_flow(frame1, frame2, model_raft, device):
    """Рассчитываем оптический поток между двумя кадрами с использованием RAFT"""
    transform = T.Compose(
        [
            T.ConvertImageDtype(torch.float32),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )

    # Преобразуем кадры в тензоры и нормализуем
    frame1_tensor = (
        transform(torch.tensor(frame1).permute(2, 0, 1) / 255.0).unsqueeze(0).to(device)
    )
    frame2_tensor = (
        transform(torch.tensor(frame2).permute(2, 0, 1) / 255.0).unsqueeze(0).to(device)
    )

    # Вычисляем оптический поток с использованием RAFT
    with torch.no_grad():
        flow = model_raft(frame1_tensor, frame2_tensor)[-1]  # Получаем оптический поток

    return flow

In [28]:
# Функция для коррекции оптического потока обратно в оригинальное разрешение
def adjust_flow_to_original_resolution(flow, original_shape, scale_x, scale_y):
    """Корректировка оптического потока в соответствии с исходным разрешением изображения"""
    # flow имеет форму (2, H, W), поэтому нужно правильно изменить его размер
    flow = torch.tensor(flow).unsqueeze(
        0
    )  # Добавляем батч, чтобы была форма (N, 2, H, W)

    # Масштабируем тензор обратно в исходное разрешение
    flow_resized = torch.nn.functional.interpolate(
        flow,
        size=(original_shape[0], original_shape[1]),
        mode="bilinear",
        align_corners=False,
    )

    # Убираем размер батча
    flow_resized = flow_resized.squeeze(0)

    # Корректируем векторы потока по осям X и Y
    flow_resized[0] *= scale_x  # Корректируем по оси X
    flow_resized[1] *= scale_y  # Корректируем по оси Y

    return flow_resized.cpu().numpy()


In [29]:
def process_frames_for_optical_flow(
    frames, model_raft, target_size=None, device="cuda"
):
    num_frames = len(frames) - 1
    original_shape = frames[0].shape[:2]  # Размер первого кадра

    # Определяем размерность массива для хранения всех оптических потоков
    optical_flows_shape = (
        num_frames + 1,
        2,
        original_shape[0],
        original_shape[1],
    )  # (количество кадров, высота, ширина, 2 канала (x и y))
    optical_flows = np.zeros(optical_flows_shape, dtype=np.float32)

    #     for i in tqdm(range(num_frames), desc="Calculating optical flow"):
    for i in range(num_frames):
        # Если указан target_size, масштабируем кадры
        if target_size:
            # Масштабируем оба кадра до target_size и вычисляем коэффициенты масштабирования
            frame1_resized, scale_x1, scale_y1 = resize_frame_with_scale(
                frames[i], target_size
            )
            frame2_resized, scale_x2, scale_y2 = resize_frame_with_scale(
                frames[i + 1], target_size
            )

            # Проверяем, что коэффициенты масштабирования для обоих кадров одинаковые
            assert (
                scale_x1 == scale_x2 and scale_y1 == scale_y2
            ), "Коэффициенты масштабирования должны совпадать!"
        else:
            # Если target_size не указан, оставляем кадры без изменений
            frame1_resized = frames[i]
            frame2_resized = frames[i + 1]
            scale_x1, scale_y1 = (
                1,
                1,
            )  # Масштабирование не производится, коэффициенты равны 1

        # Рассчитываем оптический поток между кадрами
        flow = calculate_optical_flow(
            frame1_resized, frame2_resized, model_raft, device
        )

        # Корректируем оптический поток обратно в оригинальное разрешение, если было масштабирование
        if target_size:
            flow_corrected = adjust_flow_to_original_resolution(
                flow.squeeze().cpu().numpy(), original_shape, scale_x1, scale_y1
            )
        else:
            flow_corrected = (
                flow.squeeze().cpu().numpy()
            )  # Если не было масштабирования, корректировка не нужна

        # Сохраняем оптический поток в заранее выделенный массив
        optical_flows[i] = flow_corrected

        # Освобождаем память
        del flow
        torch.cuda.empty_cache()

    return optical_flows

    

### Функция получения оптического потока по координатам скелета

In [30]:
def extract_optical_flow_at_landmarks(landmarks_data, optical_flows):
    """
    Извлекает значения скорректированного оптического потока для каждого сустава в каждом кадре.
    
    Args:
        landmarks_data: numpy массив нормализованных координат суставов (shape: [num_frames, num_joints, 3])
        optical_flows: numpy массив скорректированного оптического потока (shape: [num_frames, 2, H, W])
        
    Returns:
        flow_at_landmarks: numpy массив скорректированного оптического потока для каждого сустава
                           (shape: [num_frames, num_joints, 2])
    """
    num_frames, num_joints, _ = landmarks_data.shape
    image_height, image_width = optical_flows.shape[2], optical_flows.shape[3]
    
    flow_at_landmarks = np.zeros((num_frames, num_joints, 2), dtype=np.float32)
    
    for i in range(num_frames):
        for j in range(num_joints):
            # Преобразуем нормализованные координаты x и y в пиксели
            x = int(landmarks_data[i, j, 0] * image_width)
            y = int(landmarks_data[i, j, 1] * image_height)
            
            # Проверяем, находятся ли координаты в пределах изображения
            if 0 <= x < image_width and 0 <= y < image_height:
                # Извлекаем значения оптического потока по x и y в данной точке
                flow_x = optical_flows[i, 0, y, x]
                flow_y = optical_flows[i, 1, y, x]
                flow_at_landmarks[i, j] = [flow_x, flow_y]
            else:
                # Если координаты выходят за границы, оставляем значения (0, 0)
                flow_at_landmarks[i, j] = [0, 0]

    return flow_at_landmarks

### Класс сглаживания вектора потока

In [31]:
class OneEuroFilter:
    def __init__(self, min_cutoff=1.0, beta=0.0, d_cutoff=1.0, freq=60.0):
        self.min_cutoff = min_cutoff
        self.beta = beta
        self.d_cutoff = d_cutoff
        self.freq = freq
        self.last_value = None
        self.last_derivative = 0.0

    def alpha(self, cutoff):
        tau = 1.0 / (2.0 * np.pi * cutoff)
        return 1.0 / (1.0 + tau * self.freq)

    def filter(self, value):
        if self.last_value is None:
            self.last_value = value
            return value

        # Вычисляем производную
        derivative = (value - self.last_value) * self.freq
        alpha_d = self.alpha(self.d_cutoff)
        self.last_derivative += alpha_d * (derivative - self.last_derivative)

        # Вычисляем сглаживающее значение
        cutoff = self.min_cutoff + self.beta * np.abs(self.last_derivative)
        alpha = self.alpha(cutoff)
        smoothed_value = self.last_value + alpha * (value - self.last_value)

        # Обновляем предыдущее значение
        self.last_value = smoothed_value

        return smoothed_value


#### Функция сглаживания оптического потока всех суставов

In [32]:
def smooth_landmark_flows(flow_at_landmarks):
    """
    Применяет OneEuroFilter к каждому суставу для каждого кадра в массиве flow_at_landmarks.

    Args:
        flow_at_landmarks: numpy массив формы (num_frames, num_landmarks, 2), содержащий скорректированный оптический поток для каждого сустава в каждом кадре.
        
    Returns:
        smoothed_flows: numpy массив сглаженных значений той же формы, что и flow_at_landmarks.
    """
    # Создаем экземпляры фильтров для X и Y координат
    filters_x = [OneEuroFilter(min_cutoff=1.0, beta=0.005) for _ in range(flow_at_landmarks.shape[1])]
    filters_y = [OneEuroFilter(min_cutoff=1.0, beta=0.005) for _ in range(flow_at_landmarks.shape[1])]

    # Функция для сглаживания потока одного сустава в одном кадре
    def smooth_single_landmark(flow, idx):
        smoothed_x = filters_x[idx].filter(flow[0])
        smoothed_y = filters_y[idx].filter(flow[1])
        return np.array([smoothed_x, smoothed_y])

    # Применяем сглаживание ко всем кадрам и суставам
    num_frames, num_landmarks, _ = flow_at_landmarks.shape
    smoothed_flows = np.zeros_like(flow_at_landmarks)

    for frame_idx in range(num_frames):
        for landmark_idx in range(num_landmarks):
            smoothed_flows[frame_idx, landmark_idx] = smooth_single_landmark(
                flow_at_landmarks[frame_idx, landmark_idx], landmark_idx
            )

    return smoothed_flows

### Функции корректировка с учетом оптического потока

In [33]:
def correct_joint_coordinates(joint_coords, mean_optical_flow):
    """
    Корректирует координаты суставов на основе среднего оптического потока.

    :param joint_coords: Массив размером (33, 3), содержащий мировые координаты 33 суставов.
    :param mean_optical_flow: Массив размером (2,), содержащий средний оптический поток по осям X и Y.
    :return: Скорректированные мировые координаты суставов.
    """
    # Проверяем наличие данных о суставах (все координаты не равны нулю)
    if np.allclose(joint_coords, 0):
        # Если координаты сустава не обнаружены, возвращаем нулевой массив
        return np.zeros_like(joint_coords)

    # Проверяем, что массив имеет правильный размер
    assert joint_coords.shape == (33, 3), "joint_coords должен иметь размер (33, 3)"
    assert mean_optical_flow.shape == (2,), "mean_optical_flow должен иметь размер (2,)"

    # Извлекаем координаты 23-го (индекс 22) и 24-го (индекс 23) суставов
    joint_23 = joint_coords[22]
    joint_24 = joint_coords[23]

    # Находим расстояние между 23 и 24 суставами в мировых координатах (метры)
    distance_world_x = np.abs(joint_24[0] - joint_23[0])  # по оси X
    distance_world_y = np.abs(joint_24[1] - joint_23[1])  # по оси Y

    # Предположим, что мы имеем пиксельные расстояния для тех же суставов
    # Допустим, расстояние в пикселях между 23 и 24 суставами (пример)
    distance_image_x = 200  # Пиксельное расстояние по X (пример)
    distance_image_y = 150  # Пиксельное расстояние по Y (пример)

    # Коэффициенты преобразования (метры на пиксель)
    conversion_factor_x = distance_world_x / distance_image_x
    conversion_factor_y = distance_world_y / distance_image_y

    # Преобразуем мировые координаты в единицы изображения (пиксели)
    joint_coords_converted = np.zeros_like(joint_coords)

    # Если коэффициенты равны нулю, не делим на них, оставляем нули
    joint_coords_converted[:, 0] = np.where(
        conversion_factor_x != 0, joint_coords[:, 0] / conversion_factor_x, 0
    )
    joint_coords_converted[:, 1] = np.where(
        conversion_factor_y != 0, joint_coords[:, 1] / conversion_factor_y, 0
    )
    joint_coords_converted[:, 2] = joint_coords[
        :, 2
    ]  # Z не меняем, так как Z не учитывает оптический поток

    # Вычитаем средний оптический поток из координат в пикселях (по X и Y)
    joint_coords_converted[:, 0] -= mean_optical_flow[0]  # Корректируем X
    joint_coords_converted[:, 1] -= mean_optical_flow[1]  # Корректируем Y

    # Возвращаем скорректированные координаты в мировые единицы
    corrected_joint_coords = np.zeros_like(joint_coords)
    corrected_joint_coords[:, 0] = (
        joint_coords_converted[:, 0] * conversion_factor_x
    )  # Возвращаем X
    corrected_joint_coords[:, 1] = (
        joint_coords_converted[:, 1] * conversion_factor_y
    )  # Возвращаем Y
    corrected_joint_coords[:, 2] = joint_coords_converted[:, 2]  # Z остается неизменным

    return corrected_joint_coords

In [34]:
def correct_world_coordinates(world_landmarks_data, mean_optical_flows):
    """
    Применяет корректировку координат суставов для каждого кадра на основе среднего оптического потока.

    :param world_landmarks_data: Массив размером (num_frames, 33, 3), содержащий мировые координаты суставов для каждого кадра.
    :param mean_optical_flows: Список массивов (num_frames,) с оптическими потоками по осям X и Y для каждого кадра.
    :return: Скорректированные координаты суставов для всех кадров.
    """
    corrected_joint_coords_all_frames = []

    for i in range(len(world_landmarks_data)):
        joint_coords = world_landmarks_data[i]
        mean_optical_flow = mean_optical_flows[i]

        # Корректируем координаты суставов для каждого кадра
        corrected_coords = correct_joint_coordinates(joint_coords, mean_optical_flow)
        corrected_joint_coords_all_frames.append(corrected_coords)

    return np.array(corrected_joint_coords_all_frames)

### Функции визуализации

In [35]:
def get_sportsmen_center(landmarks_data, image_shape):
    width, height = image_shape

    # Преобразуем координаты из диапазона [0, 1] в реальные координаты на изображении
    scaled_landmarks = landmarks_data.copy()
    scaled_landmarks[:, :, 0] *= width  # Масштабируем x координаты
    scaled_landmarks[:, :, 1] *= height  # Масштабируем y координаты

    # Координаты бедер (строки 23 и 24)
    left_hip = scaled_landmarks[:, 23]  # Левая бедренная кость
    right_hip = scaled_landmarks[:, 24]  # Правая бедренная кость

    # Находим координаты середины между бедрами
    center_x = (left_hip[:, 0] + right_hip[:, 0]) / 2  # Среднее по оси x
    center_y = (left_hip[:, 1] + right_hip[:, 1]) / 2  # Среднее по оси y

    # Возвращаем координаты центра
    return center_x, center_y


In [36]:
def draw_landmarks(image, landmarks):
    image_with_landmarks = image.copy()
    height, width = image_with_landmarks.shape[:2]

    for x, y, _ in landmarks:
        # Преобразуем координаты x, y из диапазона [0, 1] в пиксельные координаты
        x_pixel = int(x * width)
        y_pixel = int(y * height)
        # Рисуем круг для каждого ключевого кадра
        cv2.circle(image_with_landmarks, (x_pixel, y_pixel), 5, (0, 255, 0), -1)

    return image_with_landmarks

In [37]:
def draw_center(image, center_x, center_y, size=25, color=(255, 255, 255)):
    """Рисует центр между бедрами на изображении."""
    # Копируем изображение, чтобы не изменять оригинал
    image_with_cross = image.copy()
    
    # Преобразуем координаты в целочисленные значения для рисования
    center_x = int(center_x)
    center_y = int(center_y)
    
    # Рисуем синий крест
    cv2.line(image_with_cross, (center_x - size, center_y), (center_x + size, center_y), color, 7)  # Горизонтальная линия
    cv2.line(image_with_cross, (center_x, center_y - size), (center_x, center_y + size), color, 7)  # Вертикальная линия
    
    return image_with_cross

In [38]:
# Функция для визуализации оптического потока
def visualize_flow_2(flow):
    """Визуализация оптического потока"""
    # Добавляем измерение для батча (N=1)
    flow = torch.tensor(flow, dtype=torch.float32).unsqueeze(0)  # Теперь форма будет (1, 2, H, W)

    # Преобразуем оптический поток в изображение
    flow_image = flow_to_image(flow)

    # Преобразуем тензор в формат numpy для визуализации
    flow_image_np = flow_image.squeeze(0).permute(1, 2, 0).cpu().numpy()

    return flow_image_np  # Возвращаем изображение для дальнейшей обработки

def visualize_flow(flow, step=4, scale=1):
    """Визуализация оптического потока с усилением векторов и возвращение изображения."""
    h, w = flow.shape[:2]
    y, x = (
        np.mgrid[step // 2 : h : step, step // 2 : w : step].reshape(2, -1).astype(int)
    )
    fx, fy = flow[y, x].T

    # Создаем пустое изображение с фоном и стрелками
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.imshow(np.zeros((h, w)), cmap="gray")  # Фон изображения

    # Увеличиваем масштаб стрелок для наглядности
    ax.quiver(x, y, fx, fy, color="r", angles="xy", scale_units="xy", scale=scale)
    ax.axis("off")
    
    # Сохраняем изображение в буфер
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close(fig)
    buf.seek(0)

    # Преобразуем буфер в изображение формата PIL, а затем в NumPy
    img = Image.open(buf)
    img_np = np.array(img)

    return img_np  # Возвращаем изображение в формате NumPy

In [39]:
def draw_crop_area(image, crop_area):
    min_x, min_y, max_x, max_y = crop_area
    # Копируем изображение, чтобы не изменять исходное
    image_with_box = image.copy()
    # Рисуем прямоугольник по координатам crop_area
    cv2.rectangle(image_with_box, (min_x, min_y), (max_x, max_y), (255, 255, 255), 5) 
    return image_with_box

#### Векторы оптического потока суставов

In [40]:
def draw_skeleton_optical_flow_arrows(image, landmarks, optical_flow, color=(255, 0, 0), scale=1, tip_length=0.15):
    """
    Рисует стрелки, представляющие векторы оптического потока для всех суставов на изображении.
    
    Args:
        image: изображение, на котором рисуются стрелки.
        landmarks: массив координат суставов (shape: [33, 2]), представляющий (x_pixel, y_pixel) координаты.
        optical_flow: оптический поток (shape: [2, H, W]), содержащий компоненты потока по x_pixel и y_pixel.
        color: цвет стрелок.
        scale: коэффициент масштабирования для увеличения длины стрелок.
    """
    image_with_arrows = image.copy()
    
    for joint in range(landmarks.shape[0]):
        x, y, _ = landmarks[joint]
        # Преобразуем координаты x_pixel, y_pixel из диапазона [0, 1] в пиксельные координаты
        x_pixel = int(x * image.shape[1])
        y_pixel = int(y * image.shape[0])
        # Проверяем, что координаты не выходят за пределы изображения и не равны нулю (паддинг)
        if 0 <= x_pixel < image.shape[1] and 0 <= y_pixel < image.shape[0] and x_pixel > 0 and y_pixel > 0:
            flow_x = optical_flow[0, int(y_pixel), int(x_pixel)]
            flow_y = optical_flow[1, int(y_pixel), int(x_pixel)]
            
            # Конечные координаты вектора с учетом масштаба
            end_x = int(x_pixel + flow_x * scale)
            end_y = int(y_pixel + flow_y * scale)
            
            # Рисуем стрелку от сустава до конечной точки
            cv2.arrowedLine(image_with_arrows, (int(x_pixel), int(y_pixel)), (end_x, end_y), color, 2, tipLength=tip_length)
    
    return image_with_arrows

#### Вектор среднего оптического потока

In [41]:
def draw_mean_optical_flow_arrows(image, center_x, center_y, flow_x, flow_y, color=(0, 255, 0), scale=1, tip_length=0.15):
    """
    Рисует стрелку, представляющую вектор оптического потока, в точке (center_x, center_y).
    
    Args:
        image: изображение, на котором рисуется стрелка.
        center_x, center_y: координаты точки, в которой рисуется вектор потока.
        flow_x, flow_y: компоненты вектора оптического потока.
        color: цвет стрелки.
        scale: коэффициент масштабирования для увеличения длины стрелки.
    """
    image_with_arrow = image.copy()
    
    # print(f"{center_x=}")
    # print(f"{center_y=}")
    # print(f"{np.any(np.isnan(flow_x))=}")
    # print(f"{np.any(np.isnan(flow_y))=}")
    
    # Конечные координаты вектора с учетом масштаба
    end_x = int(center_x + flow_x * scale)
    end_y = int(center_y + flow_y * scale)
    
    # Рисуем стрелку от центра до конечной точки
    cv2.arrowedLine(image_with_arrow, (int(center_x), int(center_y)), (end_x, end_y), color, 2, tipLength=tip_length)
    
    return image_with_arrow

#### Анимация общего оптического потока

In [42]:
import matplotlib.animation as animation
from matplotlib import pyplot as plt

def create_animation(frames, corrected_flows, center_x, center_y, save_path, name):
    fig, ax = plt.subplots(figsize=(6, 6))

    def update(frame_idx):
        ax.clear()
        frame = frames[frame_idx]
        flow_x = corrected_flows[frame_idx][0]
        flow_y = corrected_flows[frame_idx][1]

        image_with_corrected_arrow = draw_mean_optical_flow_arrows(
            frame,
            center_x[frame_idx],
            center_y[frame_idx],
            flow_x,
            flow_y,
            color=(0, 255, 0),  # Зеленая стрелка для скорректированного потока
            scale=5
        )

        ax.imshow(cv2.cvtColor(image_with_corrected_arrow, cv2.COLOR_BGR2RGB))
        ax.set_title(f"{name} (Frame {frame_idx})")
        ax.axis("off")

    ani = animation.FuncAnimation(fig, update, frames=len(frames), interval=1000)

    # Сохраняем анимацию как GIF
    ani.save(save_path, writer='pillow', fps=5)
    plt.close()


#### Анимация оптического потока скелета

In [43]:
def create_skeleton_flow_animation(frames, corrected_flows, landmarks_data, save_path, name):
    """
    Создает анимацию, отображающую векторы оптического потока для всех суставов скелета в каждом кадре.

    Args:
        frames: Массив изображений для отображения.
        corrected_flows: Массив скорректированных оптических потоков формы (num_frames, 2, H, W).
        landmarks_data: Массив координат суставов формы (num_frames, num_landmarks, 3).
        save_path: Путь для сохранения анимации.
        name: Название анимации.
    """
    fig, ax = plt.subplots(figsize=(6, 6))

    def update(frame_idx):
        ax.clear()
        frame = frames[frame_idx]
        # flow_x = corrected_flows[frame_idx][0]
        # flow_y = corrected_flows[frame_idx][1]
        # landmarks = landmarks_data[frame_idx]

        # Копируем изображение для рисования стрелок
        image_with_arrows = draw_skeleton_optical_flow_arrows(frame.copy(), landmarks_data[frame_idx], corrected_flows[frame_idx])
        ax.imshow(cv2.cvtColor(image_with_arrows, cv2.COLOR_BGR2RGB))
        ax.set_title(f"{name} (Frame {frame_idx})")
        ax.axis("off")

    ani = animation.FuncAnimation(fig, update, frames=len(frames), interval=1000)

    # Сохраняем анимацию как GIF
    ani.save(save_path, writer='pillow', fps=5)
    plt.close()

In [44]:
def create_skeleton_flow_animation2(frames, corrected_flows, landmarks_data, save_path, name):
    """
    Создает анимацию, отображающую векторы оптического потока для всех суставов скелета в каждом кадре.

    Args:
        frames: Массив изображений для отображения.
        corrected_flows: Массив скорректированных оптических потоков формы (num_frames, 2, H, W).
        landmarks_data: Массив координат суставов формы (num_frames, num_landmarks, 3).
        save_path: Путь для сохранения анимации.
        name: Название анимации.
    """
    fig, ax = plt.subplots(figsize=(6, 6))

    def update(frame_idx):
        ax.clear()
        frame = frames[frame_idx]
        flow_x = corrected_flows[frame_idx]
        flow_y = corrected_flows[frame_idx]
        landmarks = landmarks_data[frame_idx]

        image_with_arrows = frame.copy()

        # Проходим по каждому суставу и рисуем векторы оптического потока
        for i, landmark in enumerate(landmarks):
            x, y = int(landmark[0] * frame.shape[1]), int(landmark[1] * frame.shape[0])  # Преобразуем координаты в пиксели
            if 0 <= x < frame.shape[1] and 0 <= y < frame.shape[0]:
                flow_at_joint_x = flow_x[i][0]
                flow_at_joint_y = flow_y[i][1]
                image_with_arrows = draw_mean_optical_flow_arrows(image_with_arrows, x, y, flow_at_joint_x, flow_at_joint_y, color=(0, 255, 0), scale=5)

        ax.imshow(cv2.cvtColor(image_with_arrows, cv2.COLOR_BGR2RGB))
        ax.set_title(f"{name} (Frame {frame_idx})")
        ax.axis("off")

    ani = animation.FuncAnimation(fig, update, frames=len(frames), interval=1000)

    # Сохраняем анимацию как GIF
    ani.save(save_path, writer='pillow', fps=5)
    plt.close()

#### Скелет

In [45]:
# Определение связей между суставами (индексы)
skeleton_connections = [
    (0, 2),
    (0, 5),
    (2, 7),
    (5, 8),
    (5, 4),
    (5, 6),
    (2, 1),
    (2, 3),
    (10, 9),
    (11, 12),
    (12, 14),
    (14, 16),
    (16, 22),
    (16, 20),
    (20, 18),
    (18, 16),
    (11, 13),
    (13, 15),
    (15, 21),
    (15, 19),
    (19, 17),
    (17, 15),
    (12, 24),
    (11, 23),
    (23, 24),
    (24, 26),
    (26, 28),
    (28, 30),
    (30, 32),
    (32, 28),
    (23, 25),
    (25, 27),
    (27, 29),
    (29, 31),
    (31, 27),
]

In [46]:
def visualize_world_landmarks_2d(world_landmarks, frame_idx, ax=None):
    """Визуализация мировых координат в 2D (только x и y) для кадра с соединениями суставов."""
    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 8))
    
    # Получаем координаты x и y для всех точек скелета
    x_coords = world_landmarks[frame_idx, :, 0]
    # Инвертируем координаты по оси y
    y_coords = -world_landmarks[frame_idx, :, 1]

    # Отображаем точки
    ax.scatter(x_coords, y_coords, c='r', marker='o')

    # Отображаем соединения суставов
    for start, end in skeleton_connections:
        x_line = [x_coords[start], x_coords[end]]
        y_line = [y_coords[start], y_coords[end]]
        ax.plot(x_line, y_line, c='b')

    # Подписываем оси
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    ax.set_title(f'2D World Landmarks (Frame {frame_idx})')

    # Устанавливаем лимиты осей для лучшей визуализации
    ax.set_xlim([-1,1])
    ax.set_ylim([-1,1])
    ax.set_aspect('equal', adjustable='box')  # Сохраняем пропорции

    return ax


### Функция формирования полного видео

In [ ]:
def get_combined_data(combined_data, check_result):
    # Проходим по элементам исходного словаря
    for key, value in check_result.items():
        index_video = int(value['index_video'].item())

        if index_video not in combined_data:
            combined_data[index_video] = {
                'num_frames': 0,
                'num_sequences': 1,
                'skip_frames': value['skip_frames'],
                'validation_mask': value['validation_mask'],
                'mean_figure_flows': value['mean_figure_flows'],
                'flow_at_landmarks': value['flow_at_landmarks'],
                'smoothed_mean_figure_flows': value['smoothed_mean_figure_flows'],
                'smoothed_flow_at_landmarks': value['smoothed_flow_at_landmarks'],
                'landmarks_data': value['landmarks_data'],
                'world_landmarks_data': value['world_landmarks_data'],
                'corrected_world_landmarks_data': value['corrected_world_landmarks_data'],
            }
        else:
            combined_data[index_video]['num_sequences'] += 1
            combined_data[index_video]['validation_mask'] = torch.cat((combined_data[index_video]['validation_mask'], value['validation_mask']))
            combined_data[index_video]['mean_figure_flows'] = torch.cat((combined_data[index_video]['mean_figure_flows'], value['mean_figure_flows']))
            combined_data[index_video]['flow_at_landmarks'] = torch.cat((combined_data[index_video]['flow_at_landmarks'], value['flow_at_landmarks']))
            combined_data[index_video]['smoothed_mean_figure_flows'] = torch.cat((combined_data[index_video]['smoothed_mean_figure_flows'], value['smoothed_mean_figure_flows']))
            combined_data[index_video]['smoothed_flow_at_landmarks'] = torch.cat((combined_data[index_video]['smoothed_flow_at_landmarks'], value['smoothed_flow_at_landmarks']))
            combined_data[index_video]['landmarks_data'] = torch.cat((combined_data[index_video]['landmarks_data'], value['landmarks_data']))
            combined_data[index_video]['world_landmarks_data'] = torch.cat((combined_data[index_video]['world_landmarks_data'], value['world_landmarks_data']))
            combined_data[index_video]['corrected_world_landmarks_data'] = torch.cat((combined_data[index_video]['corrected_world_landmarks_data'], value['corrected_world_landmarks_data']))

        combined_data[index_video]['num_frames'] = combined_data[index_video]['validation_mask'].shape[0]
    
    return combined_data

# Формирование датасета для расчета

Это просто набор видео.

In [ ]:
class VideoDataset(Dataset):
    def __init__(self, video_ids, video_dir, max_length=50, frame_skip=3):
        self.video_ids = video_ids
        self.video_dir = video_dir
        self.max_length = max_length
        self.frame_skip = frame_skip
        # Словарь для хранения длины каждого видео
        self.video_lengths = {}
        self.sequence_counts = self._calculate_sequence_counts()

    def _calculate_sequence_counts(self):
        sequence_counts = []
        for video_id in self.video_ids:
            video_path = f"{self.video_dir}/{video_id}.mp4"
            cap = cv2.VideoCapture(video_path)
            num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

            # Рассчитываем количество последовательностей, которые можно извлечь из этого видео
            num_sequences = max(0, (num_frames + self.max_length - 1) // self.max_length)
            num_sequences = max(0, -(-num_frames // self.max_length))

            # Проверим, если ли в последней последовательности хотя бы один подходящий кадр
            if not any(frame % self.frame_skip == 0 for frame in list(range((num_sequences-1)*self.max_length, num_frames))):
                num_sequences = num_sequences - 1
                
            # Сохраняем длину видео в словарь
            self.video_lengths[video_id] = (
                num_frames,
                sum(sequence_counts),
                num_sequences,
            )
            sequence_counts.append(num_sequences)
        return sequence_counts

    def __len__(self):
        # Общее количество последовательностей во всех видео
        return sum(self.sequence_counts)

    def __getitem__(self, idx):
        # Находим к какому видео относится индекс и какая последовательность
        cumulative_count = 0
        video_index = 0

        while idx >= cumulative_count + self.sequence_counts[video_index]:
            cumulative_count += self.sequence_counts[video_index]
            video_index += 1

        video_path = f"{self.video_dir}/{self.video_ids[video_index]}.mp4"
        sequence_number = idx - cumulative_count

        cap = cv2.VideoCapture(video_path)
        num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        # Устанавливаем позицию в видео на начало последовательности
        start_frame = sequence_number * self.max_length
        end_frame = min((sequence_number + 1) * self.max_length, num_frames)
        
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        # Считываем кадры последовательности
        frames = []
        # for i in range(self.max_length):
        for i in range(start_frame, end_frame):
            # Пропускаем кадры, чтобы извлекать не каждый
            if i % self.frame_skip != 0:
                cap.grab()
                continue

            ret, frame = cap.read()
            if not ret:
                break
            frames.append(frame)

        # Добавляем один дополнительный кадр (будет использован только для расчет оптического потока)
        ret, frame = cap.read()
        if ret:
            frames.append(frame)

        cap.release()

        # Преобразуем список кадров в массив
        frames_array = np.array(frames)

        return (
            self.video_ids[video_index],
            sequence_number,
            start_frame,
            frames_array.shape[0],
            self.frame_skip,
            frames_array.shape[1],
            frames_array.shape[2],
            frames_array,
        )

# Перечни видео, используемые для работы

In [49]:
train_video = list(range(0, 200))

In [50]:
if kaggle:
    # video_dir = "/kaggle/input/figure-skating-video-analysis/video/video"
    video_dir = "/kaggle/input/fsva-custom-data"
else:
    video_dir = "../../../data/video/"
max_length = 50
frame_skip = 3

train_dataset = VideoDataset(train_video, video_dir, max_length, frame_skip)

### Создание DataLoader

In [51]:
batch_size = 1
random_seed = 50

train_videoloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

# Функции обработки видео

## "Новый" вариант

In [ ]:
def process_videos(
    dataloader,
    result_cache,
    result,
    device,
    mode,
    random_seed=50,
    pass_until=0,
    stop_time=10000,
    show=True,
    show_frames=None,
    target_size=None,
    scale_arrow=1,
    scale_factor=1.0,
    padding=100,
):
    # Устанавливаем фиксированный seed
    set_seed(random_seed)
    start_process_time = time.time()
    for k, batch in enumerate(dataloader):
        fragments = list(zip(*batch))
        try:
            if k < pass_until:
                continue
            start_calculating_time = time.time()
            for fragment_idx, fragment in enumerate(fragments):
                print(f"Fragment: {k * len(fragments) + fragment_idx} from {len(dataloader)}, mode: '{mode}'")

                key = (
                    # Номер видео
                    int(fragment[0]),
                    # Начальный кадр
                    int(fragment[3]),
                    # Количество кадров
                    int(fragment[2]),
                    # Количество пропусков кадров
                    int(fragment[4]),
                )
                hash_key = get_hash_key(key)
                if show:
                    print(hash_key)

                # Если для данного видео нет рассчитанных данных, то считаем
                if hash_key not in result_cache:
                    frames_array = fragment[7].numpy()
                    num_frames = int(fragment[3])                    
                    height = int(fragment[5])
                    width = int(fragment[6])

                    if show:
                        if show_frames is not None:
                            show_frames = [x for num in show_frames for x in (num, num + 1)]
                        else:
                            show_frames = [0, num_frames-2]

                    if show:
                        print("Calculating...")

                    start_landmarks_time = time.time()
                    # 1. Расчет скелетных данных
                    landmarks_data, world_landmarks_data, figure_masks_data = (SFeatureProcessor(model_path, key).process_video(frames_array, 25))
                    figure_masks_data = figure_masks_data.astype(np.uint8)
                    end_landmarks_time = time.time()
                    
                    # Вычисляем координаты центра между бедрами для каждого кадра
                    center_x, center_y = get_sportsmen_center(landmarks_data, (width, height))

                    # Если скелетные данные выходят за пределы изображения (координаты больше 1 или меньше 0),
                    # то кадры с такими скелетными данными помечаются как "пустые"
                    # Скелетные данные таких кадров приводятся к 0.
                    # Если больше 50% кадров нулевые или их более 10 подряд, то
                    # последовательность считается невалидной и будет пропущена при обучении
                    # и предсказании в будущем. Решение о валидности принимается
                    # до расчета оптического потока. При работе с моделью такие кадры
                    # будут помечаться, как невалидные. Если таких кадров будет много,
                    # пользователю будет возвращаться информация о невозможности обработки видео
                    start_check_landmarks_time = time.time()
                    landmarks_data, world_landmarks_data, figure_masks_data, is_valid_sequence, validation_mask = check_landmarks(landmarks_data, world_landmarks_data, figure_masks_data)
                    end_check_landmarks_time = time.time()
                    
                    if show:
                        print(f"{landmarks_data.shape=}")
                        print(f"{landmarks_data.shape=}")
                        print(f"{world_landmarks_data.shape=}")
                        print(f"{figure_masks_data.shape=}")
                        print(f"{validation_mask.shape=}")

                    if is_valid_sequence:
                        
                        if show:
                            print(f"{validation_mask=}")

                        # Вычисляем область кропа на основе координат скелета
                        crop_area, crop_shape = get_global_crop_area(landmarks_data, image_shape=(width, height), padding=padding)
                        
                        if show:
                            print(f"{crop_area=}")
                            print(f"{crop_shape=}")

                        # Отображаем изображения кадров с границами кропа, а также маску фигуры спортсмена
                        if show:
                            fig, axs = plt.subplots(1, len(show_frames), figsize=(len(show_frames)*5, 5))  
                            for i, frame in enumerate(show_frames):
                                # Рисуем границы crop на изображении
                                image_with_box = draw_crop_area(frames_array[frame], crop_area)
                                # Рисуем landmarks на изображении
                                image_with_landmarks = draw_landmarks(image_with_box, landmarks_data[frame])
                                # Рисуем центр между бедрами на изображении
                                image_with_center = draw_center(image_with_landmarks, center_x[frame], center_y[frame])
                                axs[i].imshow(cv2.cvtColor(image_with_center, cv2.COLOR_BGR2RGB))
                                axs[i].set_title(f"Image {frame} with landmarks")
                                axs[i].axis("off")
                            plt.show()

                            fig, axs = plt.subplots(1, len(show_frames), figsize=(len(show_frames)*5, 5))  
                            for i, frame in enumerate(show_frames):
                                mask_with_box = draw_crop_area(figure_masks_data[frame]*255, crop_area)
                                axs[i].imshow(cv2.cvtColor(mask_with_box, cv2.COLOR_BGR2RGB))
                                axs[i].set_title(f"Mask {frame} with crop area")
                                axs[i].axis("off")
                            plt.show()

                        # Применяем кроп ко всем изображениям
                        padded_images, padding_masks = crop_with_padding(frames_array, crop_area)
                        padded_figure_masks, _ = crop_with_padding(figure_masks_data, crop_area)

                        # Корректируем координаты скелета
                        padded_landmarks_data = np.array([padding_landmarks(landmarks_data[i], crop_area, (width, height)) for i in range(len(landmarks_data))])
                        # Вычисляем координаты центра между бедрами для каждого кадра
                        padded_center_x, padded_center_y = get_sportsmen_center(padded_landmarks_data, padded_images[0].shape[:2])

                        # Отображаем изображения кадров с паддингом с границами кропа, а также маску фигуры спортсмена
                        if show:
                            fig, axs = plt.subplots(1, len(show_frames), figsize=(len(show_frames)*5, 5))  
                            for i, frame in enumerate(show_frames):
                                # Рисуем landmarks на изображении
                                image_with_landmarks = draw_landmarks(padded_images[frame], padded_landmarks_data[frame])
                                # Рисуем центр между бедрами на изображении
                                image_with_center = draw_center(image_with_landmarks, padded_center_x[frame], padded_center_y[frame])
                                axs[i].imshow(cv2.cvtColor(image_with_center, cv2.COLOR_BGR2RGB))
                                axs[i].set_title(f"Padded image {frame} with landmarks")
                                axs[i].axis("off")
                            plt.show()

                            fig, axs = plt.subplots(1, len(show_frames), figsize=(len(show_frames)*5, 5))  
                            for i, frame in enumerate(show_frames):
                                # Рисуем landmarks на изображении
                                image_with_landmarks = draw_landmarks(padding_masks[frame]*255, padded_landmarks_data[frame])
                                # Рисуем центр между бедрами на изображении
                                image_with_center = draw_center(image_with_landmarks, padded_center_x[frame], padded_center_y[frame])
                                axs[i].imshow(cv2.cvtColor(image_with_center, cv2.COLOR_BGR2RGB))
                                axs[i].set_title(f"Crop mask {frame} with landmarks")
                                axs[i].axis("off")
                            plt.show()

                            fig, axs = plt.subplots(1, len(show_frames), figsize=(len(show_frames)*5, 5))  
                            for i, frame in enumerate(show_frames):
                                axs[i].imshow(cv2.cvtColor(padded_figure_masks[frame]*255, cv2.COLOR_BGR2RGB))
                                axs[i].set_title(f"Padded mask {frame} with crop area")
                                axs[i].axis("off")
                            plt.show()

                        # 2. Расчет оптического потока
                        optical_flows = process_frames_for_optical_flow(
                            padded_images,
                            model_raft,
                            target_size=target_size,
                            device=device,
                        )

                        # В последовательности пока не учитывается первый кадр следующей последовательности
                        # После добавления max_length+1 кадра - не используется
                        # optical_flows[-1] = optical_flows[-2]


                        # Исключаем последний кадр, который используется только для расчета оптического потока
                        landmarks_data, world_landmarks_data, figure_masks_data, validation_mask = (
                            landmarks_data[:-1], world_landmarks_data[:-1], figure_masks_data[:-1], validation_mask[:-1]
                        )
                        optical_flows = optical_flows[:-1]
                        padded_images, padding_masks = padded_images[:-1], padding_masks[:-1]
                        padded_figure_masks = padded_figure_masks[:-1]
                        padded_landmarks_data = padded_landmarks_data[:-1]
                        padded_center_x, padded_center_y = padded_center_x[:-1], padded_center_y[:-1]

                        # Проверяем наличие nan значений
                        if show:
                            print(f"{padded_images.shape=}")
                            print(f"{padding_masks.shape=}")
                            print(f"{padded_figure_masks.shape=}")
                            print(f"{padded_landmarks_data.shape=}")
                            print(f"{padded_center_x.shape=}")
                            print(f"{padded_center_y.shape=}")
                            print(f"{optical_flows.shape=}")
                            print(f"{np.any(np.isnan(optical_flows))=}")
                            
                        # Устраняем nan значения, если таковые появились
                        optical_flows = np.nan_to_num(optical_flows, nan=0.0)

                        if show:
                            fig, axs = plt.subplots(1, len(show_frames), figsize=(len(show_frames)*5, 5))  
                            for i, frame in enumerate(show_frames):
                                # Отображаем визуализацию оптического потока
                                flow_image_np = visualize_flow_2(optical_flows[frame])
                                axs[i].imshow(flow_image_np)
                                axs[i].set_title(f"Optical Flow {frame}")
                                axs[i].axis("off")
                            plt.show()

                            # Создаем 3 подграфика в одном ряду
                            fig, axs = plt.subplots(1, len(show_frames), figsize=(len(show_frames)*5, 5))  
                            for i, frame in enumerate(show_frames):
                                # Отображаем визуализацию векторов оптического потока
                                flow_arrows = torch.tensor(optical_flows[frame], dtype=torch.float32).unsqueeze(0)
                                flow_arrows = flow_arrows[:, :, :, :].permute(2, 3, 1, 0).squeeze().cpu().numpy()
                                flow_image_np = visualize_flow(flow_arrows, step=16, scale=1.0)
                                axs[i].imshow(flow_image_np)
                                axs[i].set_title(f"Optical Flow {frame}")
                                axs[i].axis("off")
                            plt.show()

                            fig, axs = plt.subplots(1, len(show_frames), figsize=(len(show_frames)*5, 5))  
                            for i, frame in enumerate(show_frames):
                                # Рисуем landmarks на изображении
                                image_with_landmarks = draw_landmarks(padded_images[frame], padded_landmarks_data[frame])
                                # Рисуем центр между бедрами на изображении
                                image_with_center = draw_center(image_with_landmarks, padded_center_x[frame], padded_center_y[frame])
                                # Рисуем векторы оптического потока для всех суставов на изображении
                                image_with_arrows = draw_skeleton_optical_flow_arrows(image_with_center, padded_landmarks_data[frame], optical_flows[frame])
                                axs[i].imshow(cv2.cvtColor(image_with_arrows, cv2.COLOR_BGR2RGB))
                                axs[i].set_title(f"Skeleton main OFlow {frame} frame")
                                axs[i].axis("off")
                            plt.show()

                        # 3. Расчет среднего оптического потока фона (учитывается паддинг)
                        # Инвертируем маску фигуры спортсмена (0 - для фигуриста, 1 - для фона)
                        background_masks = (padded_figure_masks == 0).astype(np.uint8)
                        
                        # Применяем маску padding_masks, чтобы исключить паддинг из учета (0 - паддинг, 1 - изображение)
                        # Объединяем обе маски: фон + изображение (без паддинга)
                        combined_masks = background_masks * padding_masks

                        if show:
                            fig, axs = plt.subplots(1, len(show_frames), figsize=(len(show_frames)*5, 5))  
                            for i, frame in enumerate(show_frames):
                                axs[i].imshow(cv2.cvtColor(combined_masks[frame]*255, cv2.COLOR_BGR2RGB))
                                axs[i].set_title(f"OFlow background mask frame {frame}")
                                axs[i].axis("off")
                            plt.show()

                        # Извлекаем компоненты оптического потока по осям X и Y
                        flow_x = optical_flows[:, 0]  # Компонента по оси X, shape: (num_frames, H, W)
                        flow_y = optical_flows[:, 1]  # Компонента по оси Y, shape: (num_frames, H, W)

                        # Применяем объединенную маску к компонентам оптического потока и вычисляем средние значения
                        mean_background_flow_x = np.sum(flow_x * combined_masks, axis=(1, 2)) / np.sum(combined_masks, axis=(1, 2))
                        mean_background_flow_y = np.sum(flow_y * combined_masks, axis=(1, 2)) / np.sum(combined_masks, axis=(1, 2))

                        # Объединяем компоненты X и Y в массив средних значений для всех кадров
                        # shape: (num_frames, 2)
                        mean_background_flows = np.stack([mean_background_flow_x, mean_background_flow_y], axis=1)  

                        # Проверяем наличие nan значений
                        if show:
                            print(f"{np.any(np.isnan(mean_background_flows))=}")
                        # Устраняем nan значения, если таковые появились
                        mean_background_flows = np.nan_to_num(mean_background_flows, nan=0.0)
                            
                        # 4. Вычитание среднего оптического потока фона (устранение движения камеры)

                        # Расширяем mean_background_flows по пространственным измерениям (H, W) до размера (17, 2, 1392, 1392)
                        mean_background_flows_expanded = mean_background_flows[:, :, np.newaxis, np.newaxis]

                        # Вычитаем средний поток фона из каждого кадра
                        corrected_optical_flows = optical_flows - mean_background_flows_expanded
                        
                        # Проверяем наличие nan значений
                        if show:
                            print(f"{np.any(np.isnan(corrected_optical_flows))=}")
                        # Устраняем nan значения, если таковые появились
                        corrected_optical_flows = np.nan_to_num(corrected_optical_flows, nan=0.0)

                        if show:
                            print(f"{mean_background_flows[0]=}")
                            print(f"{optical_flows[0][0]=}")
                            print(f"{corrected_optical_flows[0][0]=}")

                        
                        if show:
                            print(f"{padded_landmarks_data.shape=}")
                            print(f"{corrected_optical_flows.shape=}")

                        flow_at_landmarks = extract_optical_flow_at_landmarks(padded_landmarks_data, corrected_optical_flows)
                        
                        if show:
                            print(f"{flow_at_landmarks.shape=}")

                        # Проверяем наличие nan значений
                        if show:
                            print(f"{np.any(np.isnan(flow_at_landmarks))=}")

                        if show:
                            fig, axs = plt.subplots(1, len(show_frames), figsize=(len(show_frames)*5, 5))  
                            for i, frame in enumerate(show_frames):
                                # Отображаем визуализацию оптического потока
                                flow_image_np = visualize_flow_2(corrected_optical_flows[frame])
                                axs[i].imshow(flow_image_np)
                                axs[i].set_title(f"Corrected optical Flow {frame}")
                                axs[i].axis("off")
                            plt.show()

                            # Создаем 3 подграфика в одном ряду
                            fig, axs = plt.subplots(1, len(show_frames), figsize=(len(show_frames)*5, 5))  
                            for i, frame in enumerate(show_frames):
                                # Отображаем визуализацию векторов оптического потока
                                flow_arrows = torch.tensor(corrected_optical_flows[frame], dtype=torch.float32).unsqueeze(0)
                                flow_arrows = flow_arrows[:, :, :, :].permute(2, 3, 1, 0).squeeze().cpu().numpy()
                                flow_image_np = visualize_flow(flow_arrows, step=16, scale=1.0)
                                axs[i].imshow(flow_image_np)
                                axs[i].set_title(f"Corrected optical Flow {frame}")
                                axs[i].axis("off")
                            plt.show()

                            fig, axs = plt.subplots(1, len(show_frames), figsize=(len(show_frames)*5, 5))  
                            for i, frame in enumerate(show_frames):
                                # Рисуем landmarks на изображении
                                image_with_landmarks = draw_landmarks(padded_images[frame], padded_landmarks_data[frame])
                                # Рисуем центр между бедрами на изображении
                                image_with_center = draw_center(image_with_landmarks, padded_center_x[frame], padded_center_y[frame])
                                # Рисуем векторы оптического потока для всех суставов на изображении
                                image_with_arrows = draw_skeleton_optical_flow_arrows(image_with_center, padded_landmarks_data[frame], corrected_optical_flows[frame])
                                axs[i].imshow(cv2.cvtColor(image_with_arrows, cv2.COLOR_BGR2RGB))
                                axs[i].set_title(f"Skeleton main corrected OFlow {frame}")
                                axs[i].axis("off")
                            plt.show()

                        # 5. Расчет среднего оптического потока фигуры (учитывается паддинг)
                        # Инвертируем маску фигуры спортсмена (0 - для фигуриста, 1 - для фона)
                        firure_masks = (padded_figure_masks == 1).astype(np.uint8)
                        
                        # Применяем маску padding_masks, чтобы исключить паддинг из учета (0 - паддинг, 1 - изображение)
                        # Объединяем обе маски: фон + изображение (без паддинга)
                        combined_masks = firure_masks * padding_masks

                        if show:
                            fig, axs = plt.subplots(1, len(show_frames), figsize=(len(show_frames)*5, 5))  
                            for i, frame in enumerate(show_frames):
                                axs[i].imshow(cv2.cvtColor(combined_masks[frame]*255, cv2.COLOR_BGR2RGB))
                                axs[i].set_title(f"OFlow background mask frame {frame}")
                                axs[i].axis("off")
                            plt.show()

                        # Извлекаем компоненты оптического потока по осям X и Y
                        flow_x = corrected_optical_flows[:, 0]  # Компонента по оси X, shape: (num_frames, H, W)
                        flow_y = corrected_optical_flows[:, 1]  # Компонента по оси Y, shape: (num_frames, H, W)

                        mask_sum = np.sum(combined_masks, axis=(1, 2))
                        # print(f"{mask_sum=}")

                        # Инициализируем средние значения как массивы нулей
                        mean_figure_flow_x = np.zeros_like(mask_sum, dtype=float)
                        mean_figure_flow_y = np.zeros_like(mask_sum, dtype=float)

                        # Обрабатываем только те кадры, где сумма маски больше 0
                        valid_indices = mask_sum > 0

                        # Вычисляем средние значения только для валидных индексов
                        mean_figure_flow_x[valid_indices] = np.sum(flow_x[valid_indices] * combined_masks[valid_indices], axis=(1, 2)) / mask_sum[valid_indices]
                        mean_figure_flow_y[valid_indices] = np.sum(flow_y[valid_indices] * combined_masks[valid_indices], axis=(1, 2)) / mask_sum[valid_indices]

                        # Объединяем компоненты X и Y в массив средних значений для всех кадров
                        # shape: (num_frames, 2)
                        mean_figure_flows = np.stack([mean_figure_flow_x, mean_figure_flow_y], axis=1)  

                        # Проверяем наличие nan значений
                        if show:
                            print(f"{np.any(np.isnan(mean_figure_flows))=}")
                        # Устраняем nan значения, если таковые появились
                        mean_figure_flows = np.nan_to_num(mean_figure_flows, nan=0.0)
                        
                        if show:
                            print(f"{mean_figure_flows.shape=}")

                        if show:
                            fig, axs = plt.subplots(1, len(show_frames), figsize=(len(show_frames)*5, 5))  
                            for i, frame in enumerate(show_frames):
                                # Рисуем центр между бедрами на изображении
                                image_with_center = draw_center(padded_images[frame], padded_center_x[frame], padded_center_y[frame])
                                # Рисуем вектор среднего оптического потока
                                image_with_mean_arrow = draw_mean_optical_flow_arrows(
                                    image_with_center,
                                    padded_center_x[frame],
                                    padded_center_y[frame],
                                    mean_figure_flows[frame][0],
                                    mean_figure_flows[frame][1],
                                    color=(255, 0, 0),
                                    scale=scale_arrow
                                )
                                axs[i].imshow(cv2.cvtColor(image_with_mean_arrow, cv2.COLOR_BGR2RGB))
                                axs[i].set_title(f"Mean figure corrected OFlow {frame}")
                                axs[i].axis("off")
                            plt.show()

                        # 6. Расчет сглаженного скорректированного среднего оптического потока фигуры
                        # Функция для сглаживания потока одного кадра
                        # Создаем фильтры для сглаживания оптического потока по осям X и Y
                        filter_x = OneEuroFilter(min_cutoff=1.0, beta=0.005)
                        filter_y = OneEuroFilter(min_cutoff=1.0, beta=0.005)

                        def smooth_single_flow(flow):
                            smoothed_x = filter_x.filter(flow[0])
                            smoothed_y = filter_y.filter(flow[1])
                            return np.array([smoothed_x, smoothed_y])

                        # Применяем сглаживание ко всем кадрам сразу с помощью np.apply_along_axis
                        smoothed_mean_figure_flows = np.apply_along_axis(smooth_single_flow, 1, mean_figure_flows)
                        
                        if show:
                            print(f"{smoothed_mean_figure_flows.shape=}")
                        # Проверяем наличие nan значений
                        if show:
                            print(f"{np.any(np.isnan(smoothed_mean_figure_flows))=}")

                        # 7. Расчет сглаженного скорректированного оптического потока каждого сустава
                        smoothed_flow_at_landmarks = smooth_landmark_flows(flow_at_landmarks)

                        if show:
                            print(f"{smoothed_flow_at_landmarks.shape=}")
                        # Проверяем наличие nan значений
                        if show:
                            print(f"{np.any(np.isnan(smoothed_flow_at_landmarks))=}")

                        if show:
                            # Cоздание и сохранения анимации среднего оптического потока фигуры
                            create_animation(padded_images, mean_figure_flows, padded_center_x, padded_center_y, save_path="mean_sportsmen_flows.gif", name="Mean Figure Optical Flow",)
                            # Cоздание и сохранения анимации сглаженного среднего оптического потока фигуры
                            create_animation(padded_images, smoothed_mean_figure_flows, padded_center_x, padded_center_y, save_path="smoothed_mean_sportsmen_flows.gif", name="Smoothed Mean Figure Optical Flow",)
                            # Cоздание и сохранения анимации оптического потока скелета
                            create_skeleton_flow_animation(padded_images, corrected_optical_flows, padded_landmarks_data, 'skeleton_flow_animation.gif', 'Skeleton Flow Animation')
                            # Cоздание и сохранения анимации сглаженного оптического потока скелета
                            create_skeleton_flow_animation2(padded_images, smoothed_flow_at_landmarks, padded_landmarks_data, 'smoothed_skeleton_flow_animation.gif', 'Smoothed Skeleton Flow Animation')


                        # 7. Корректирока мировых координат
                        corrected_world_landmarks_data = correct_world_coordinates(world_landmarks_data, mean_background_flows)

                        # Визуализация не скорректированных мировых координат - 2D
                        if show:
                            fig, axs = plt.subplots(1, len(show_frames), figsize=(len(show_frames)*5, 5))  
                            for i, frame in enumerate(show_frames):
                                visualize_world_landmarks_2d(corrected_world_landmarks_data, frame, ax=axs[i])

                            plt.show()
                            
                        # 9. Сохранение результатов расчета
                        result[hash_key] = {
                            # Номер видео
                            'index_video': fragment[0],
                            # Начальный кадр
                            'start_frame': fragment[2],
                            # Количество кадров
                            'num_frames': fragment[3],
                            # Количество пропусков кадров
                            'skip_frames': fragment[4],
                            # Флаг валидности последовательности
                            'is_valid_sequence': is_valid_sequence,
                            # Маска валидности
                            'validation_mask': torch.tensor(validation_mask, dtype=torch.uint8).cpu(),
                            # Скорректированный средний оптический поток фигуры
                            'mean_figure_flows': torch.tensor(mean_figure_flows, dtype=torch.float16).cpu(),
                            # Скорректированный оптический поток скелета
                            'flow_at_landmarks': torch.tensor(flow_at_landmarks, dtype=torch.float16).cpu(),
                            # Сглаженный скорректированный средний оптический поток фигуры
                            'smoothed_mean_figure_flows': torch.tensor(smoothed_mean_figure_flows, dtype=torch.float16).cpu(),
                            # Сглаженный скорректированный оптический поток скелета
                            'smoothed_flow_at_landmarks': torch.tensor(smoothed_flow_at_landmarks, dtype=torch.float16).cpu(),
                            # Оригинальные координаты скелета
                            'landmarks_data': torch.tensor(landmarks_data, dtype=torch.float16).cpu(),
                            # Оригинальные мировые координаты
                            'world_landmarks_data': torch.tensor(world_landmarks_data, dtype=torch.float16).cpu(),
                            # Скорректированные мировые координаты
                            'corrected_world_landmarks_data': torch.tensor(corrected_world_landmarks_data, dtype=torch.float16).cpu(),
                        }
                        
                        if show:
                            print()
                    else:
                        print('Not valid sequence')
                        # 9. Сохранение результатов расчета
                        result[hash_key] = {
                            # Номер видео
                            'index_video': fragment[0],
                            # Начальный кадр
                            'start_frame': fragment[2],
                            # Количество кадров
                            'num_frames': fragment[3],
                            # Количество пропусков кадров
                            'skip_frames': fragment[4],
                            # Флаг валидности последовательности
                            'is_valid_sequence': is_valid_sequence,
                            # Маска валидности
                            'validation_mask': torch.tensor(np.zeros(fragment[3]), dtype=torch.uint8).cpu(),
                            # Скорректированный средний оптический поток фигуры
                            'mean_figure_flows': torch.tensor(np.zeros((fragment[3], 2)), dtype=torch.float16),
                            # Скорректированный оптический поток скелета
                            'flow_at_landmarks': torch.tensor(np.zeros((fragment[3], 33, 2)), dtype=torch.float16),
                            # Сглаженный скорректированный средний оптический поток фигуры
                            'smoothed_mean_figure_flows': torch.tensor(np.zeros((fragment[3], 2)), dtype=torch.float16),
                            # Сглаженный скорректированный оптический поток скелета
                            'smoothed_flow_at_landmarks': torch.tensor(np.zeros((fragment[3], 33, 2)), dtype=torch.float16),
                            # Оригинальные координаты скелета
                            'landmarks_data': torch.tensor(np.zeros((fragment[3], 33, 3)), dtype=torch.float16),
                            # Оригинальные мировые координаты
                            'world_landmarks_data': torch.tensor(np.zeros((fragment[3], 33, 3)), dtype=torch.float16),
                            # Скорректированные мировые координаты
                            'corrected_world_landmarks_data': torch.tensor(np.zeros((fragment[3], 33, 3)), dtype=torch.float16),
                        }
            
            end_calculating_time = time.time()

            print(f"Calculating time: {end_calculating_time - start_calculating_time}")
            print(f"Landmarks time: {end_landmarks_time - start_landmarks_time}")
            print(f"Check_landmarks time: {end_check_landmarks_time - start_check_landmarks_time}")
            
        except Exception as e:
            print(f"Get Exception: {e}. Continue")
            traceback.print_exc()
            print('Error sequence')
            # 9. Сохранение результатов расчета
            result[hash_key] = {
                # Номер видео
                'index_video': fragment[0],
                # Начальный кадр
                'start_frame': fragment[2],
                # Количество кадров
                'num_frames': fragment[3],
                # Количество пропусков кадров
                'skip_frames': fragment[4],
                # Флаг валидности последовательности
                'is_valid_sequence': is_valid_sequence,
                # Маска валидности
                'validation_mask': torch.tensor(np.zeros(fragment[3]), dtype=torch.uint8).cpu(),
                # Скорректированный средний оптический поток фигуры
                'mean_figure_flows': torch.tensor(np.zeros((fragment[3], 2)), dtype=torch.float16),
                # Скорректированный оптический поток скелета
                'flow_at_landmarks': torch.tensor(np.zeros((fragment[3], 33, 2)), dtype=torch.float16),
                # Сглаженный скорректированный средний оптический поток фигуры
                'smoothed_mean_figure_flows': torch.tensor(np.zeros((fragment[3], 2)), dtype=torch.float16),
                # Сглаженный скорректированный оптический поток скелета
                'smoothed_flow_at_landmarks': torch.tensor(np.zeros((fragment[3], 33, 2)), dtype=torch.float16),
                # Оригинальные координаты скелета
                'landmarks_data': torch.tensor(np.zeros((fragment[3], 33, 3)), dtype=torch.float16),
                # Оригинальные мировые координаты
                'world_landmarks_data': torch.tensor(np.zeros((fragment[3], 33, 3)), dtype=torch.float16),
                # Скорректированные мировые координаты
                'corrected_world_landmarks_data': torch.tensor(np.zeros((fragment[3], 33, 3)), dtype=torch.float16),
            }
            
            continue

        # Время, затраченное на обучение
        process_time = time.time() - start_process_time

        # Ранняя остановка для kaggle с учетом ограничений по времени
        if process_time > stop_time:
            print("Early stoping by time")
            break

    return "Done"

# Обработка обучающих видео

In [53]:
%%time

# Создаем кэш для эмбеддингов
train_cache = []
train_result = {}
# Обрабатываем видео с использованием DataLoader
process_videos(
    train_videoloader,
    train_cache,
    train_result,
    device,
    mode="train",
    random_seed=random_seed,
    pass_until=0,
    stop_time=42000,
    show=False,
    show_frames=(0, 6),
    target_size=(128, 128),
    scale_arrow=2,
    padding=100
);

Fragment: 0 from 610, mode: 'train'


I0000 00:00:1730661043.316594      17 task_runner.cc:85] GPU suport is not available: INTERNAL: ; RET_CHECK failure (mediapipe/gpu/gl_context_egl.cc:77) display != EGL_NO_DISPLAYeglGetDisplay() returned error 0x300c
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1730661043.447028     104 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730661043.507043     103 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730661043.591388     102 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Calculating time: 1.366452932357788
Landmarks time: 1.3588383197784424
Check_landmarks time: 0.0014133453369140625
Fragment: 1 from 610, mode: 'train'


I0000 00:00:1730661044.960732      17 task_runner.cc:85] GPU suport is not available: INTERNAL: ; RET_CHECK failure (mediapipe/gpu/gl_context_egl.cc:77) display != EGL_NO_DISPLAYeglGetDisplay() returned error 0x300c
W0000 00:00:1730661045.082940     117 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730661045.132883     117 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Calculating time: 1.2636570930480957
Landmarks time: 1.2593588829040527
Check_landmarks time: 0.001220703125
Fragment: 2 from 610, mode: 'train'


...

Calculating time: 1.290947437286377
Landmarks time: 1.2865476608276367
Check_landmarks time: 0.00029397010803222656
Fragment: 608 from 610, mode: 'train'


I0000 00:00:1730662162.932646      17 task_runner.cc:85] GPU suport is not available: INTERNAL: ; RET_CHECK failure (mediapipe/gpu/gl_context_egl.cc:77) display != EGL_NO_DISPLAYeglGetDisplay() returned error 0x300c
W0000 00:00:1730662163.041324    4974 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730662163.066083    4972 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Calculating time: 1.1544594764709473
Landmarks time: 1.150423288345337
Check_landmarks time: 0.0002758502960205078
Fragment: 609 from 610, mode: 'train'


I0000 00:00:1730662164.219009      17 task_runner.cc:85] GPU suport is not available: INTERNAL: ; RET_CHECK failure (mediapipe/gpu/gl_context_egl.cc:77) display != EGL_NO_DISPLAYeglGetDisplay() returned error 0x300c
W0000 00:00:1730662164.317003    4981 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730662164.341295    4981 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Calculating time: 0.5584414005279541
Landmarks time: 0.5547363758087158
Check_landmarks time: 0.000171661376953125
CPU times: user 30min 56s, sys: 2min 23s, total: 33min 20s
Wall time: 18min 41s


In [54]:
# Инициализируем результирующий словарь
train_combined_data = {}
train_combined_data = get_combined_data(train_combined_data, train_result)

In [55]:
if kaggle:
    torch.save(train_combined_data, "/kaggle/working/calculating_data.pt")
else:
    torch.save(train_combined_data, "working/calculating_data.pt")